#Setup

In [1]:
#GET Data
from google.colab import files
files.upload() #Upload kaggle.json here.

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!mkdir aisegment-split-1
%cd aisegment-split-1
!kaggle datasets download -d vincenttu/aisegment-split-1
!unzip -q aisegment-split-1.zip
!rm aisegment-split-1.zip
%cd ..

/content/aisegment-split-1
100% 3.56G/3.56G [00:27<00:00, 136MB/s]
100% 3.56G/3.56G [00:27<00:00, 137MB/s]
/content


In [37]:
#Imports 
import PIL
from glob import glob
import torch 
from tqdm.notebook import tqdm
from torch import nn
import pandas as pd
import torch.optim as optim 
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms as T
import torch.nn.functional as F

! pip install segmentation-models-pytorch -qqq
! pip install git+https://github.com/qubvel/segmentation_models.pytorch -qqq
import segmentation_models_pytorch as smp

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


#Pre-processing

In [55]:
filepaths = list(glob('/content/aisegment-split-1/*'))
#filepaths

In [56]:
#Extract data and label sets
label_paths = []
image_paths = []
for path in filepaths:
  if 'label' in path:
    label_paths.append(path)
  else:
    image_paths.append(path)
  
for x in label_paths:
  y = x.split("_")[1]
  y = '/content/aisegment-split-1/' + y
  if y not in image_paths:
    label_paths.remove(x)

In [57]:
label_paths.remove('/content/aisegment-split-1/label_1803201916-00000002.jpg')
image_paths.remove('/content/aisegment-split-1/1803201916-00000002.jpg')

In [58]:
print(len(label_paths))
print(len(image_paths))

label_paths.sort()
image_paths.sort()

8605
8605


In [59]:
label_paths[0], image_paths[0]

('/content/aisegment-split-1/label_1803151818-00019737.jpg',
 '/content/aisegment-split-1/1803151818-00019737.jpg')

In [60]:
from torchvision.transforms.transforms import ToTensor
class SegmentDataset(Dataset):
  def __init__(self, images_path, labels_path):
    self.images_path = images_path
    self.labels_path = labels_path

    self.transform_both = T.Compose([
        T.ToTensor(),            
        T.Resize((256, 256)) #want to make square, messes up ration but all good     
    ])

    self.transform_image = T.Compose([
        T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    self.transform_label = T.Compose([
                            
    ])

  def __len__(self):
    return len(self.images_path)

  def __getitem__(self, idx):
    image = PIL.Image.open(self.images_path[idx])
    label = PIL.Image.open(self.labels_path[idx])

    image_array = np.asarray(image)

    #make into array, get only alpha color channel, expand to make 3D(with C.C.)
    label_array = np.expand_dims(np.asarray(label)[:,:,3], axis=2)
    
    image_tensor = self.transform_both(image_array)
    label_tensor = self.transform_both(label_array)

    #only transform images (training set) cause it goes through model
    image_tensor = self.transform_image(image_tensor)
    label_tensor = self.transform_label(label_tensor)

    label_tensor = (label_tensor>0).type(torch.FloatTensor)  # np.float32, float


    # print("label_tensor_transform: ", label_tensor.shape)
    # print("image tensor transform: ", image_tensor.shape)

    return image_tensor, label_tensor

#PRINT Shape, Datatype, range

In [61]:
# from matplotlib import pyplot as plt
# plt.imshow(train_data[8605][0].permute(1, 2, 0).squeeze())

In [62]:
# plt.imshow(train_data[8605][1].permute(1, 2, 0).squeeze())

# Model Creation

In [63]:
##MODEL installation(UNET)

model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

# Model training

In [69]:
#Variable instatiation
batch_size = 64
training_data = SegmentDataset(image_paths, label_paths)

train_loader = DataLoader(training_data, 
                          batch_size=batch_size, 
                          shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

cool_model = model.train().to(device)

num_epochs = 1
learning_rate = 0.001

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [68]:
torch.cuda.empty_cache()

In [70]:
#Training Loop
for epoch in range(num_epochs):
    for i, batch in tqdm(enumerate(train_loader)):
        image, label = batch 
        image = image.to(device)
        label = label.to(device)

        # Forward pass
        outputs = cool_model(image)

        loss = F.binary_cross_entropy(torch.sigmoid(outputs), label)
        
        optimizer.zero_grad()
        # Backward and optimize
        loss.backward()


    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))   

0it [00:00, ?it/s]

Epoch [1/1], Loss: 0.7635


In [71]:
torch.cuda.get_device_name()

'Tesla T4'